### GOLD LAYER – Banking Analytics
Gold layer provides aggregated, business-ready datasets for BI tools, dashboards, and reporting.

##### Input (ONLY from Silver)
- apextrust_catalog.silver.customers
- apextrust_catalog.silver.accounts
- apextrust_catalog.silver.transactions
- apextrust_catalog.silver.branches


#### Gold Notebook – Setup

In [0]:
%sql

USE CATALOG apextrust_catalog;
CREATE SCHEMA IF NOT EXISTS gold;


Read Silver Tables (Delta only)

In [0]:
customers_df = spark.read.table("apextrust_catalog.silver.customers")
accounts_df = spark.read.table("apextrust_catalog.silver.accounts")
transactions_df = spark.read.table("apextrust_catalog.silver.transactions")
branches_df = spark.read.table("apextrust_catalog.silver.branches")

##### Total Balance by City


In [0]:
from pyspark.sql.functions import sum

gold_balance_city = (
    customers_df
    .join(accounts_df, "customer_id")
    .groupBy("city")
    .agg(sum("balance").alias("total_balance"))
)

gold_balance_city.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("apextrust_catalog.gold.balance_by_city")

display(gold_balance_city)

#### Debit vs Credit Volume


In [0]:
from pyspark.sql.functions import sum

gold_txn_type = (
    transactions_df
    .groupBy("txn_type")
    .agg(sum("amount").alias("total_amount"))
)

gold_txn_type.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("apextrust_catalog.gold.debit_credit_volume")

display(gold_txn_type);


#### Daily Transaction Value

In [0]:
from pyspark.sql.functions import sum

gold_daily_txn = (
    transactions_df
    .groupBy("txn_date")
    .agg(sum("amount").alias("daily_txn_value"))
)

gold_daily_txn.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("apextrust_catalog.gold.daily_transactions")

display(gold_daily_txn);

#### Regional Banking Summary (EXTRA VALUE)

In [0]:
gold_region_summary = (
    branches_df
    .join(customers_df, "city")
    .join(accounts_df, "customer_id")
    .groupBy("region")
    .agg(
        sum("balance").alias("total_balance"),
    )
)

gold_region_summary.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("apextrust_catalog.gold.region_summary")

display(gold_region_summary);

- Gold = aggregations
- Gold tables = Delta tables
- Used by BI tools
- No business rules here (already applied in Silver)
